# level: Hard

# Description

The demons had captured the princess (P) and imprisoned her in the bottom-right corner of a dungeon. The dungeon consists of M x N rooms laid out in a 2D grid. Our valiant knight (K) was initially positioned in the top-left room and must fight his way through the dungeon to rescue the princess.

The knight has an initial health point represented by a positive integer. If at any point his health point drops to 0 or below, he dies immediately.

Some of the rooms are guarded by demons, so the knight loses health (negative integers) upon entering these rooms; other rooms are either empty (0's) or contain magic orbs that increase the knight's health (positive integers).

In order to reach the princess as quickly as possible, the knight decides to move only rightward or downward in each step.

 

Write a function to determine the knight's minimum initial health so that he is able to rescue the princess.

For example, given the dungeon below, the initial health of the knight must be at least 7 if he follows the optimal path RIGHT-> RIGHT -> DOWN -> DOWN.

惡魔抓住了公主(P) 並將其監禁在地城(dungeon) 之右下角.
地城有 M x N 個房間構成二維盤面.
一位英勇的騎士(K) 從左上方進入並且中戰鬥並救出公主.

騎士有以正整數表示的初始的血量數值, 只要在任一時刻其血量降到 0 或更低, 他就會立刻死亡.

有些房間有惡魔駐守, 所以會使騎士在進入房間之後受傷(負整數); 其他房間可能是空的(0) 或著是有回復道具使騎士回血(正整數).

為了盡快到達公主所在地, 騎士決定每次都只走向右或向下.

寫下一個函數決定騎士要解救公主所需之最少血量.

以下面的地城為例, 騎士一開始需要有最少七點血量如果他走最佳路徑 右 -> 右 -> 下 -> 下

||||
|-|-|-|
|-2 (K)|-3|3|
|-5|-10|1|
|10|30|-5 (P)|

# Note:
* The knight's health has no upper bound.
* Any room can contain threats or power-ups, even the first room the knight enters and the bottom-right room where the princess is imprisoned.

* 騎士的血量沒有上限
* 每個房間都有威脅或助益, 包含騎士最初的房間以及公主被監禁處

# 想法

1. 從最右下往回推, 每一格都計算他要進入這一格所需之最小血量
> 例子的最後一格會扣五點血(-5) 所以要進入這一格之前騎士需要有 x + (-5) = 1 => 最少要有 6 點血
2. 進入[1, 2] 前之血量則是 x + 1 = 6 => x = 5
> * 進入 [1, 2] 前血量大於等於 5 滴血的話, 進入後血量大於等於 6 滴血, 所以以此條件進入 [1, 3] 會沒事
> * 同理, 進入 [2, 1] 的條件是 x + 30 = 6 => x = -24, 但是因為血量不得小於 1, 所以改用 1 代替
> * 換言之, 進入第 [i, j] 格之條件為 x = max(1, min(dungeon[i + 1, j], dungeon[i, j + 1]) - dungeon[i, j])
> * 如果沒有第 [i +1, j] 或 [i, j + 1] 格, 則只取有的那一項, 如果兩者都沒有, 則用最終值 1 代替

# 實作

先建立一個上下左右顛倒的順序的處理矩陣, 每一個都是進入對應格之前的最低血量需求
然後逐步往前一格推, 最後就可以得到初始之最低血量需求

In [1]:
def calculateMinimumHP(dungeon):
    process = []
    for l in dungeon[-1::-1]:
        tmp = []
        for x in l[-1::-1]:
            if not process and not tmp:
                # 最後一格
                tmp.append(max(1 - x, 1))

            elif not tmp:
                # 沒有[i, j + 1]
                tmp.append(max(1,process[-1][0] - x))

            elif not process:
                # 沒有 [i + 1, j]:
                tmp.append(max(1,tmp[-1] - x))

            else:
                # 有 [i + 1, j] 也有 [i, j + 1]
                tmp.append(max(1, min(tmp[-1] - x, process[-1][len(tmp)] - x)))

        process.append(tmp)
    return process[-1][-1]

In [2]:
dungeon = [[-2,-3,3],[-5,-10,1],[10,30,-5]]
calculateMinimumHP(dungeon)

7

# 二版

其實不需要儲存每一個位置的資訊, 每次只需儲存前一 row 之結果

In [3]:
def calculateMinimumHP(dungeon):
    process = []
    for l in dungeon[-1::-1]:
        tmp = []
        for x in l[-1::-1]:
            if not process and not tmp:
                # 最後一格
                tmp.append(max(1 - x, 1))

            elif not tmp:
                # 沒有[i, j + 1]
                tmp.append(max(1,process[0] - x))

            elif not process:
                # 沒有 [i + 1, j]:
                tmp.append(max(1,tmp[-1] - x))

            else:
                # 有 [i + 1, j] 也有 [i, j + 1]
                tmp.append(max(1, min(tmp[-1] - x, process[len(tmp)] - x)))

        process = tmp
    return process[-1]

In [4]:
dungeon = [[-2,-3,3],[-5,-10,1],[10,30,-5]]
calculateMinimumHP(dungeon)

7

# 三版
修改修改 process 之方式, 因此不需要 tmp

In [1]:
def calculateMinimumHP(dungeon):
    process = []
    for l in dungeon[-1::-1]:
        for j, x in enumerate(l[-1::-1]):
            if not process:
                # 最後一格
                process.append(max(1 - x, 1))

            elif j == 0:
                # 沒有[i, j + 1]
                process[j] = max(1,process[j] - x)

            elif len(process) < len(dungeon[0]):
                # 沒有 [i + 1, j]:
                process.append(max(1,process[j -1] - x))

            else:
                # 有 [i + 1, j] 也有 [i, j + 1]
                process[j] = max(1, min(process[j - 1], process[j]) - x)
    return process[-1]

In [2]:
dungeon = [[-2,-3,3],[-5,-10,1],[10,30,-5]]
calculateMinimumHP(dungeon)

7